## Getting the dataset from csv file

Imports

In [18]:
import io
import numpy as np
import pandas as pd
import math

Create dataframe from CSV file

In [ ]:
df = pd.read_csv(r'FoDS-Assignment-2.csv')
df = df.sample(frac = 1)
df.to_csv('FoDS-Assignment-2_Shuffled.csv')
df

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,sqft_living15,sqft_lot15,price
411,3,2.00,1940.0,6440,1.0,0,2,5,7,970.0,970,1730,4640,651000.0
197,5,2.75,3300.0,7560,2.0,0,0,3,10,3300.0,0,3150,8580,1110000.0
944,2,1.00,1480.0,6210,1.0,0,0,3,7,1080.0,400,1290,7509,331000.0
179,4,2.00,2290.0,6000,NaN,0,3,5,7,2290.0,0,2000,3150,685000.0
1003,6,2.75,2730.0,5000,1.0,0,0,3,8,1780.0,950,2090,5000,733000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,4,2.50,1940.0,7107,2.0,0,0,3,7,1940.0,0,1850,5705,322500.0
338,3,1.75,1680.0,9769,1.0,0,0,3,8,1340.0,340,1730,9686,265000.0
587,5,1.75,2220.0,6000,1.5,0,0,3,7,1420.0,800,1550,1715,607500.0
1173,3,1.75,1460.0,7980,1.0,0,0,3,7,1460.0,0,1920,7980,290000.0


## Pre-processing

Handling missing data

In [ ]:
null_counts = df.isnull().sum()
null_counts.drop([index for index in null_counts.index if null_counts[index] == 0], inplace=True)
print("The features having missing data and number of empty rows:\n")
print(null_counts)
print("---")

index_list = []
for feature in null_counts.index:
  index_list.append(df[df[feature].isnull()].index)

print(index_list)

# print the specific rows of df with NaN values for each feature
for index in index_list:
  print(df.loc[index].to_string())

df.fillna(df.mean(skipna=True, numeric_only=True), inplace=True)

# print the specific rows of df after replacing NaN values for each feature
for index in index_list:
  print(df.loc[index].to_string())

The features having missing data and number of empty rows:

sqft_living    14
floors         13
sqft_above     14
dtype: int64
---
[Int64Index([145, 838, 85, 906, 62, 958, 316, 153, 380, 551, 408, 34, 782, 828], dtype='int64'), Int64Index([179, 223, 725, 665, 185, 72, 345, 102, 392, 620, 100, 476, 559], dtype='int64'), Int64Index([31, 335, 323, 337, 248, 1154, 752, 748, 1125, 321, 1024, 794, 690,
            173],
           dtype='int64')]
     bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  view  condition  grade  sqft_above  sqft_basement  sqft_living15  sqft_lot15     price
145         4       4.00          NaN      7221     2.0           0     0          3     10      3010.0              0           3490        7518  765000.0
838         3       2.25          NaN      3500     1.5           0     0          5      8      1540.0            300           1800        3300  785200.0
85          4       2.50          NaN      7930     2.0           0     0          3   

Normalisation

In [ ]:
df=(df-df.mean())/df.std()
df

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,sqft_living15,sqft_lot15,price
411,-0.411804,-0.183538,-0.192788,-0.151543,-9.523292e-01,-0.058099,2.283950,2.390923,-0.585225,-0.990067,1.456095,-0.421707,-0.306532,0.255831
197,1.738628,0.778415,1.227097,-0.133405,9.157623e-01,-0.058099,-0.320016,-0.646956,1.967703,1.736886,-0.665835,1.648058,-0.160138,1.446861
944,-1.487021,-1.466142,-0.673043,-0.155268,-9.523292e-01,-0.058099,-0.320016,-0.646956,-0.585225,-0.861326,0.209188,-1.063043,-0.199932,-0.574517
179,0.663412,-0.183538,0.172624,-0.158669,8.295993e-16,-0.058099,3.585933,2.390923,-0.585225,0.554816,-0.665835,-0.028160,-0.361894,0.344055
1003,2.813845,0.778415,0.631998,-0.174864,-9.523292e-01,-0.058099,-0.320016,-0.646956,0.265751,-0.042070,1.412344,0.103022,-0.293156,0.468608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,0.663412,0.457764,-0.192788,-0.140741,9.157623e-01,-0.058099,-0.320016,-0.646956,-0.585225,0.145188,-0.665835,-0.246797,-0.266961,-0.596573
338,-0.411804,-0.504189,-0.464236,-0.097631,-9.523292e-01,-0.058099,-0.320016,-0.646956,0.265751,-0.557031,0.077934,-0.421707,-0.119044,-0.745777
587,1.738628,-0.504189,0.099541,-0.158669,-1.828345e-02,-0.058099,-0.320016,-0.646956,-0.585225,-0.463402,1.084210,-0.684072,-0.415213,0.142955
1173,-0.411804,-0.504189,-0.693923,-0.126603,-9.523292e-01,-0.058099,-0.320016,-0.646956,-0.585225,-0.416587,-0.665835,-0.144767,-0.182432,-0.680906


Test-train splitting: 70-30

In [ ]:
train_data = df.iloc[:833]
test_data = df.iloc[833:]

train_data = train_data.to_numpy()
test_data = test_data.to_numpy()

## Model

Input for the model

In [ ]:
def input_function():
  number_of_features = int(input("Enter number of features in dataset: "))
  n = int(input("Enter number of iterations in Gradient Descent: "))
  eta = float(input("Enter the learning rate(eta): "))
  return (n, eta, number_of_features)

Linear Regression with Gradient Descent

In [ ]:
def gradientDescent(index_array, no_of_iterations, eta):
  data_train = np.concatenate((np.ones(train_data.shape[0], dtype=np.float64).reshape(-1,1), train_data[:, index_array], (-1)*(train_data[:, [-1]])), axis = 1)
  data_test = np.concatenate((np.ones(test_data.shape[0], dtype=np.float64).reshape(-1,1), test_data[:, index_array], (-1)*(test_data[:, [-1]])), axis = 1)
  n_coeffs = 1 + len(index_array)
  coefficients = np.ones(n_coeffs+1).reshape(-1,1) # Initializing coefficient matrix
  coefficients[n_coeffs] = 1 # To consider the pressure value

  for _ in range(no_of_iterations):      
    # Matrix Multiplying coeffcients with data points
    intermediate_matrix = data_train.dot(coefficients)
    slope = np.zeros(n_coeffs)
    j = 0
    while(j < n_coeffs):
        slope[j] = float((intermediate_matrix*data_train[:, [j]]).sum())
        j = j+1
    slope = slope.reshape(-1,1) # Making it a column
    coefficients[:-1, :] = coefficients[:-1, :] - (eta)*slope # Gradient Descent Formula

  train_error_intermediate = data_train.dot(coefficients)
  train_error_intermediate = train_error_intermediate * train_error_intermediate
  training_error = train_error_intermediate.sum()
  test_error_intermediate = data_test.dot(coefficients)
  test_error_intermediate = test_error_intermediate * test_error_intermediate
  testing_error = test_error_intermediate.sum()
  return (training_error, testing_error)

## Linear Regression model without an pre-processing and feature selection

In [20]:
df_raw = pd.read_csv(r'FoDS-Assignment-2.csv')
df_raw = df_raw.sample(frac = 1)

train_data_raw = df_raw.iloc[:833]
test_data_raw = df_raw.iloc[833:]

train_data_raw = train_data_raw.to_numpy()
test_data_raw = test_data_raw.to_numpy()

def raw_gradientDescent(index_array, no_of_iterations, eta):
  data_train = np.concatenate((np.ones(train_data_raw.shape[0], dtype=np.float64).reshape(-1,1), train_data_raw[:, index_array], (-1)*(train_data_raw[:, [-1]])), axis = 1)
  data_test = np.concatenate((np.ones(test_data_raw.shape[0], dtype=np.float64).reshape(-1,1), test_data_raw[:, index_array], (-1)*(test_data_raw[:, [-1]])), axis = 1)
  n_coeffs = 1 + len(index_array)
  coefficients = np.ones(n_coeffs+1).reshape(-1,1) # Initializing coefficient matrix
  coefficients[n_coeffs] = 1 # To consider the pressure value

  for _ in range(no_of_iterations):      
    # Matrix Multiplying coeffcients with data points
    intermediate_matrix = data_train.dot(coefficients)
    slope = np.zeros(n_coeffs)
    j = 0
    while(j < n_coeffs):
        slope[j] = float((intermediate_matrix*data_train[:, [j]]).sum())
        j = j+1
    slope = slope.reshape(-1,1) # Making it a column
    coefficients[:-1, :] = coefficients[:-1, :] - (eta)*slope # Gradient Descent Formula

  train_error_intermediate = data_train.dot(coefficients)
  train_error_intermediate = train_error_intermediate * train_error_intermediate
  training_error = train_error_intermediate.sum()
  test_error_intermediate = data_test.dot(coefficients)
  test_error_intermediate = test_error_intermediate * test_error_intermediate
  testing_error = test_error_intermediate.sum()
  return (training_error, testing_error)

number_of_iterations, eta, number_of_features = input_function()
raw_result = raw_gradientDescent(list(range(number_of_features)), number_of_iterations, eta)

print('The training error and testing error respectively for raw Linear Regression without any pre-processing and feature selection:\n')
print(raw_result)

Enter number of features in dataset: 13
Enter number of iterations in Gradient Descent: 10000
Enter the learning rate(eta): 0.00000000000001
The training error and testing error respectively for raw Linear Regression without any pre-processing and feature selection:

(nan, nan)


## Greedy algorithms for feature selection

Forward feature selection

In [ ]:
def forwardFeatureSelection():
  number_of_iterations, eta, number_of_features = input_function()
  final_pair = [[], math.inf, math.inf]
  previous_pair = [[], math.inf]
  for i in range(number_of_features):
    current_pair = [[], math.inf]
    train_error = 0
    for j in [k for k in range(number_of_features) if k not in previous_pair[0]]:
      current_indices = [*(previous_pair[0]), j]
      temp_train_error, error = gradientDescent(current_indices, number_of_iterations, eta)
      print(current_indices, error)
      if error < current_pair[1]:
        current_pair = [current_indices, error]
        train_error = temp_train_error
      
    if current_pair[1] < final_pair[1]:
      final_pair = [*current_pair, train_error]
    previous_pair = current_pair
  return final_pair

Subset of features that provide optimal model for forward feature selection and the error when trained with only that subset of features

In [ ]:
forward_result = forwardFeatureSelection()
print("The features which give optimal model with forward feature selection, testing error and training error for those features:\n")
print(df.columns[forward_result[0]], forward_result[1:])

Enter number of features in dataset: 13
Enter number of iterations in Gradient Descent: 200000
Enter the learning rate(eta): 0.0004
[0] 247.48491325631892
[1] 220.31567619300003
[2] 139.83543812933146
[3] 270.1433522903684
[4] 240.5967901576069
[5] 242.309824393491
[6] 203.86062513977117
[7] 264.22906593754664
[8] 154.61224057543677
[9] 171.79569605486606
[10] 235.37039950385793
[11] 176.93456968007058
[12] 262.6862286434039
[2, 0] 136.76369643062185
[2, 1] 140.76165911615516
[2, 3] 146.0205190564227
[2, 4] 140.19842391372214
[2, 5] 133.81534606460593
[2, 6] 122.50121960071979
[2, 7] 137.99875622080322
[2, 8] 134.69379799120475
[2, 9] 139.88879273945582
[2, 10] 139.63148436973063
[2, 11] 139.09363239287052
[2, 12] 142.29329074672282
[2, 6, 0] 121.9860338926803
[2, 6, 1] 123.74918234863335
[2, 6, 3] 126.5040796369599
[2, 6, 4] 122.47443733522027
[2, 6, 5] 122.86536866490206
[2, 6, 7] 121.56997308043682
[2, 6, 8] 118.46084933286274
[2, 6, 9] 121.67859936136311
[2, 6, 10] 122.689840287131

Backward feature selection

In [ ]:
def backwardFeatureSelection():
  number_of_iterations, eta, number_of_features = input_function()
  final_pair = [[], math.inf, math.inf]
  previous_pair = [list(range(number_of_features)), math.inf]
  train_error, previous_pair[1] = gradientDescent(previous_pair[0], number_of_iterations, eta)
  final_pair = [*previous_pair, train_error]

  print(previous_pair)
  for i in range(number_of_features):
    current_pair = [[], math.inf]
    train_error = 0
    for j in previous_pair[0]:
      current_indices = previous_pair[0].copy()
      current_indices.remove(j)
      temp_train_error, error = gradientDescent(current_indices, number_of_iterations, eta)
      print(current_indices, error)
      if error < current_pair[1]:
        current_pair = [current_indices, error]
        train_error = temp_train_error
    if current_pair[1] < final_pair[1]:
      final_pair = [*current_pair, train_error]
    previous_pair = current_pair
  return final_pair

Subset of features that provide optimal model for backward feature selection and the errors when trained with only that subset of features

In [ ]:
backward_result = backwardFeatureSelection()
print("The features which give optimal model with backward feature selection, testing error and training error for those features:\n")
print(df.columns[backward_result[0]], backward_result[1:])

Enter number of features in dataset: 13
Enter number of iterations in Gradient Descent: 10000
Enter the learning rate(eta): 0.0004
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 126.21231664423951]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] 123.32316736817245
[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] 124.26709692450919
[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] 110.78358521396407
[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12] 123.02728441081732
[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12] 119.32679536525264
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12] 126.82278551812465
[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12] 133.03632426103957
[0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12] 129.75308384271298
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12] 130.66631610778603
[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12] 124.76990444444718
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12] 123.57491675137474
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12] 125.49493286247625
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 128.1410566374664
[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] 110.